# チートシート（LeRobot v0.4.0）

- [Installation][3]
- [SO-101][2]
- [Imitation Learning on Real-World Robots][1]

[1]: https://huggingface.co/docs/lerobot/en/il_robots
[2]: https://huggingface.co/docs/lerobot/en/so101
[3]: https://huggingface.co/docs/lerobot/en/installation

### Conda環境の作成

- [macOS/Linux](https://docs.conda.io/projects/conda/en/latest/user-guide/install/macos.html#installing-in-silent-mode)
- [Windows](https://docs.conda.io/projects/conda/en/latest/user-guide/install/windows.html#installing-on-windows)

In [ ]:
if False:
    # macOSかLinux
    !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.sh -O ~/miniconda.sh
    !bash ~/miniconda.sh -b -p $HOME/miniconda
    !source $HOME/miniconda/bin/activate
    !conda init --all
    !conda create -y -n lerobot-v0.4.0 python=3.10
    !conda activate lerobot-v0.4.0
    !conda install ffmpeg -c conda-forge
    if not os.path.exists("lerobot"):
        !git clone https://github.com/huggingface/lerobot.git
    %pip install -qe lerobot
    %pip install -qe "lerobot[feetech]" # モーターのドライバー

### USBポートの特定

In [ ]:
if False:
    # Conda環境を設定する
    !conda activate lerobot-v0.4.0

    # 1. マイコンボードをUSBで接続する
    # 2. find_portコマンドを実行
    !lerobot-find-port

    # 3. USBを外して、エンターキーを押す
    # 4. 「/dev/tty.usbmodem5A7A0182121」のようなポートが出力されるので変数に保存する
    !export LEADER_PORT="/dev/tty.usbmodem5A7A0182121"
    !export FOLLOWER_PORT="/dev/tty.usbmodem5A7A0182121"

### キャリブレーション

In [ ]:
if False:
    # IDを設定する
    !export LEADER_ID="leader_arm_1"
    !export FOLLOWER_ID="follower_arm_1"

    # Followerのキャリブレーション
    !lerobot-calibrate \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID

    # Leaderのキャリブレーション
    !lerobot-calibrate \
        --teleop.type=so101_leader \
        --teleop.port=$LEADER_PORT \
        --teleop.id=$LEADER_ID

フォロワーアームの中間位置:

![](image/follower.png)

### テレオペレーション

In [ ]:
if False:
    !lerobot-teleoperate \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --teleop.type=so101_leader \
        --teleop.port=$LEADER_PORT \
        --teleop.id=$LEADER_ID

### カメラの検出

In [ ]:
if False:
    !lerobot-find-cameras opencv
    !export CAMERAS="{ image.top: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30}, image.wrist.left: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: 30}}"

### カメラ付きテレオペレーション

In [ ]:
if False:
    !lerobot-teleoperate \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --robot.cameras=$CAMERAS \
        --teleop.type=so101_leader \
        --teleop.port=$LEADER_PORT \
        --teleop.id=$LEADER_ID \
        --display_data=true

### データセットの記録

- 実行中のキーボードコントロール
    - 右矢印: 現在のエピソードを早期終了し次へ進む
    - 左矢印: 現在のエピソードをキャンセルし再記録する
    - Escape: セッションを終了し、データセットをアップロードする
- `--resume=true`で中断した記録を再開できる
- 再開する場合、`--dataset.num_episodes`は追加で記録するエピソード数を設定する
- 記録をやり直すにはデータセットディレクトリを手動で削除する

In [ ]:
if False:
    !export HF_USER="EngineerCafeJP"
    !export DATE=$(date +%Y-%m-%d-%H-%M-%S)
    !export REPO_ID="${HF_USER}/record-test_${DATE}"
    !export NUM_EPISODES=50
    !export SINGLE_TASK="Grab the green bottle cap and place it in the brown box."

    !lerobot-record \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --robot.cameras=$CAMERAS \
        --teleop.type=so101_leader \
        --teleop.port=$LEADER_PORT \
        --teleop.id=$LEADER_ID \
        --display_data=true \
        --dataset.repo_id=$REPO_ID \
        --dataset.single_task=$SINGLE_TASK \
        --dataset.num_episodes=$NUM_EPISODES \
        --resume=false

### データセットの可視化

https://huggingface.co/spaces/lerobot/visualize_dataset

![](image/preview.png)

### エピソードの再生

In [ ]:
if False:
    !lerobot-replay \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.id=$FOLLOWER_ID \
        --dataset.repo_id=$REPO_ID \
        --dataset.episode=0

### 訓練の準備

In [ ]:
import os

try:
    from google.colab import userdata
    HF_USER = userdata.get("HF_USER")
    HF_TOKEN = userdata.get("HF_TOKEN")
    WANDB_API_KEY = userdata.get("WANDB_API_KEY")

    !pip install -q condacolab
    import condacolab
    condacolab.install()
except:
    from dotenv import load_dotenv
    load_dotenv()
    HF_USER = os.getenv("HF_USER")
    HF_TOKEN = os.getenv("HF_TOKEN")
    WANDB_API_KEY = os.getenv("WANDB_API_KEY")

assert HF_USER is not None, "HF_USER is not set"
assert HF_TOKEN is not None, "HF_TOKEN is not set"
assert WANDB_API_KEY is not None, "WANDB_API_KEY is not set"

if not os.path.exists("lerobot"):
    !git clone https://github.com/huggingface/lerobot.git
    !conda install ffmpeg=7.1.1 -c conda-forge
    !cd lerobot && pip install -e .

import wandb
wandb.login(key=WANDB_API_KEY)

### 訓練

- `policy-device=mps`でAppleシリコンでの訓練が可能

In [ ]:
# A100の場合はbatch_size=64

!export OUTPUT_DIR="outputs/train/act_so101_test"
!export JOB_NAME="act_so101_test"
!eport POLICY_NAME="act_so101_test_${DATE}"

!cd lerobot && python src/lerobot/scripts/lerobot_train.py \
    --dataset.repo_id=$REPO_ID \
    --policy.type=act \
    --output_dir=$OUTPUT_DIR \
    --job_name=act-test_2025-10-25-15-32-04 \
    --policy.device=cuda \
    --policy.push_to_hub=true \
    --policy.repo_id=$HF_USER/act-test_2025-10-25-15-32-04 \
    --wandb.enable=true \
    --num_workers=4 \
    --batch_size=64 \
    --steps=100_000 \
    --save_freq=1000 \
    --wandb.enable=true

![](image/loss.png)

![](image/l1_loss.png)

### 訓練の再開

In [ ]:
if False:
    !lerobot-train \
        --config_path=${OUTPUT_DIR}/checkpoints/last/pretrained_model/train_config.json \
        --resume=true

### チェックポイントのアップロード

In [ ]:
if False:
    !export CHECKPOINT_NAME="act_so101_test"
    !huggingface-cli upload ${HF_USER}/${CHECKPOINT_NAME} \
    ${OUTPUT_DIR}/checkpoints/last/pretrained_model

In [ ]:
if False:
    !export CHECKPOINT_NAME="act-so101-test_${DATE}"
    !hf upload \
        --token $HF_TOKEN \
        ${HF_USER}/${CHECKPOINT_NAME} \
        ${OUTPUT_DIR}/checkpoints/last/pretrained_model \
        .

In [ ]:
if False:
    !export CKPT=010000
    !huggingface-cli upload ${HF_USER}/${CHECKPOINT_NAME}_${CKPT} \
        ${OUTPUT_DIR}/checkpoints/${CKPT}/pretrained_model

### Google Colab

https://huggingface.co/docs/lerobot/en/notebooks#training-act

### 推論の実行

In [ ]:
if False:
    !export EVAL_NAME="eval-so101-test_$(date +%Y-%m-%d-%H-%M-%S)"
    !lerobot-record  \
        --robot.type=so101_follower \
        --robot.port=$FOLLOWER_PORT \
        --robot.cameras=$CAMERAS \
        --robot.id=$FOLLOWER_ID \
        --display_data=false \
        --dataset.repo_id=${HF_USER}/${EVAL_NAME} \
        --dataset.single_task=${SINGLE_TASK} \
        --policy.path=${HF_USER}/${POLICY_NAME}